**¿Cómo cambia el gasto promedio en copagos a medida que los beneficiarios envejecen?**

Para la pregunta importante que hagan:  
1. Contexto y por que queremos conocer la respuesta  
2. Por que elegimos esas variables y que representan  
3. Codigo  
4. Graficos   
5. Respuesta  

https://docs.google.com/document/d/1R72ZKTbxHIYD-pZi2cijGSqUsZ4KzQ1N0nl0RQyObv8/edit?tab=t.0

El tiempo no pasa factura, es sabido que personas de la 3ra edad suelen ir con mayor frecuencia al medico y por implicancia sus gastos medicos deben aumentar, pero la pregunta es... Que tanto aumentan estos gastos, existe un punto de quiebre o es gradual?.  
Más aun, se puede predecir el gasto promedio medico en Chile segun la edad de personas adscritas a FONASA?. Si pensamos en agrupar segun region, genero, tramo de fonasa, etc... existen diferencias clave?.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

with open("Beneficiarios Innominada 2023/export MLE 2023.csv", encoding='utf-8', errors='ignore') as f:
    benef_2023_df = pd.read_csv(f, nrows=1000)
with open("Beneficiarios Innominada 2023/Beneficiarios Fonasa 2023.csv", encoding='utf-8', errors='ignore') as f:
    mle_2023_df = pd.read_csv(f, nrows=1000)

In [5]:
mle_2023_df = mle_2023_df.rename(columns={'PRESTRACIONES': 'PRESTACIONES'})


In [ ]:
benef_2023_df

In [ ]:
mle_2023_df